In [1]:
import pandas as pd
import numpy as np

In [2]:
lpz = pd.read_csv('vendas_modificado.csv')

In [3]:
lpz.head()

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"R$ 16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"R$ 3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"R$ 1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,"R$ 8,74",4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,"R$ 9,48",2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


In [4]:
print(lpz.dtypes)

id_da_compra      int64
data             object
hora             object
cliente          object
produto          object
valor            object
quantidade        int64
total           float64
status           object
cidade           object
estado           object
pais             object
cep              object
frete           float64
pagamento        object
vendedor         object
marca            object
dtype: object


## <p align="center"><strong><span style="color:green;"> Verificação e Padronização de Tipos e Formatos</span></strong></p>

#### <p align="center"><strong>FORMATAÇÃO DE DATA E HORA</strong></p>

In [5]:
lpz['data'] = pd.to_datetime(lpz['data'], errors='coerce').dt.strftime('%Y-%m-%d')

In [6]:
lpz['hora'] = pd.to_datetime(lpz['hora'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')

In [7]:
print(lpz[['data', 'hora']].head())

         data      hora
0  2021-03-20  23:35:51
1  2020-10-30  09:00:53
2  2021-06-09  15:30:28
3  2022-06-04  08:41:23
4  2019-05-04  13:38:45


#### <p align="center"><strong>VALORES NUMÉRICOS</strong></p>

In [8]:
# Convertendo colunas para formato numérico
colunas_numericas = ['quantidade', 'total', 'frete']

for coluna in colunas_numericas:
    lpz[coluna] = pd.to_numeric(lpz[coluna], errors='coerce')

# Verificando os tipos de dados após a conversão
print(lpz.dtypes)

id_da_compra      int64
data             object
hora             object
cliente          object
produto          object
valor            object
quantidade        int64
total           float64
status           object
cidade           object
estado           object
pais             object
cep              object
frete           float64
pagamento        object
vendedor         object
marca            object
dtype: object


In [9]:
lpz.to_csv('vendas_corrigido.csv', index=False)

## <p align="center"><strong><span style="color:purple;">Tratamento de Inconsistências</span></strong></p>

!! Antes de iniciar o tratamento de valores ausentes irei corrijir a coluna produto pois pode ajudar com os valores ausentes !!

In [10]:
print(lpz[['produto']].head())

            produto
0  Queijo Mussarela
1   Molho de Tomate
2      Água Mineral
3            Carvão
4              Café


Importação da biblioteca (re) para trabalhar com expressões regulares 

In [11]:
import re

In [12]:
# Remover caracteres especiais da coluna 'produto'
lpz['produto'] = lpz['produto'].astype(str).apply(
    lambda x: re.sub(r'[^A-Za-z0-9À-ÿ\s]', '', x)
)

In [13]:
print(lpz[['produto']].head())

            produto
0  Queijo Mussarela
1   Molho de Tomate
2      Água Mineral
3            Carvão
4              Café


In [14]:
lpz.to_csv('vendas_corrigido.csv', index=False)

#### <p align="center"><strong>VALORES AUSENTES</strong></p>

In [15]:
print(lpz.isnull().sum())

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3685
status             0
cidade             0
estado             0
pais               0
cep                0
frete           7371
pagamento          0
vendedor        3680
marca              0
dtype: int64


VALORES FALTANTES E ESCOLHA DO TRATAMENTO DEPENDENDO DA COLUNA<br>
TORTAL: RECALCULAR DE VALOR,QUANTIDADE E FRETE SE ESTIVEREM CORRETOS <br>
FRETE: MEDIA <br>
VENDEDOR: REMOÇÃO OU APLICAR A PALAVRA DESCONHECIDO <br>

##### <p align="center"><strong>TRATAMENTO DE VALOR</strong></p>

In [17]:
lpz['valor'] = lpz['valor'].str.replace(r'R\$', '', regex=True).str.strip()

In [18]:
lpz.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba


##### <p align="center"><strong>TRATAMENTO DO FRETE</strong></p>

In [19]:
lpz['Frete_Check1'] = lpz.groupby(['cidade', 'estado'])['frete'].transform('median')

In [20]:
lpz.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,Frete_Check1
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,0.0
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,0.0
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba,22.0


In [21]:
lpz['Frete_Check2'] = lpz.frete.fillna(lpz.groupby (['cidade', 'estado'])['frete'].transform('median'))

In [22]:
lpz.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,Frete_Check1,Frete_Check2
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,0.0,20.0
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,0.0,0.0
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba,22.0,22.0


In [23]:
lpz['frete'] =   lpz.frete.fillna(lpz.groupby (['cidade', 'estado'])['frete'].transform('median'))

In [24]:
lpz = lpz.drop(columns=['Frete_Check1', 'Frete_Check2'])

In [25]:
lpz.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba


In [26]:
print(lpz.isnull().sum())

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3685
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor        3680
marca              0
dtype: int64


In [27]:
lpz.to_csv('vendas_corrigido.csv', index=False)

##### <p align="center"><strong>VERIFICAÇÃO DO VALOR TOTAL</strong></p>

In [28]:
def corrigir_valor(valor):
    match = re.search(r'\d{1,3}(,\d{2})?', str(valor))  # pega o primeiro número com vírgula decimal
    if match:
        return match.group().replace(',', '.')
    return np.nan

lpz['valor'] = lpz['valor'].apply(corrigir_valor).astype(float)

In [29]:
# Calcular o total correto
lpz['total_calculado'] = lpz['valor'] * lpz['quantidade'] + lpz['frete']

In [30]:
lpz[lpz.total.isnull()].head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_calculado
320,15092,2018-03-30,21:29:12,GUILHERME HUGO CAZULA PEREIRA,Feijão,10.19,2,NaN,Em Separação,Itaboraí,RJ,Brasil,24800-000,0.0,Cartão de Crédito,GABRIEL QUEIROZ DE AGUIAR,Caldo Bom,20.38
410,33369,2020-08-21,09:10:58,GABRIEL PAULO ALVES ALVIM,Refrigerante,6.21,2,NaN,Em Transporte,Angra dos Reis,RJ,Brasil,23900-000,0.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Coca-Cola,12.42
448,28894,2022-01-09,15:55:37,THIAGO HENRIQUE CAÇADOR,Macarrão,2.49,14,NaN,Em Transporte,Cataguases,MG,Brasil,36770-000,0.0,Pix,LUCAS VITOR FAÇANHA NEVES,Renata,34.86
561,15983,2022-10-26,17:31:51,LUCAS RISSON EUZEBIO,Café,8.01,6,NaN,Em Separação,Astolfo Dutra,MG,Brasil,36780-000,0.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações,48.06
565,38996,2022-10-15,14:36:22,THIAGO CARDOSO DINIZ,Café,12.13,2,NaN,Pagamento Confirmado,São Bernardo do Campo,SP,Brasil,09700-000,0.0,Cartão de Crédito,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Melitta,24.26
580,21272,2020-09-05,10:45:01,PAULO SEGURASSE GONÇALVES,Sabonete,2.55,7,NaN,PC,Simão Pereira,MG,Brasil,36123-000,9.5,Cartão de Crédito,PAULO SOUZA RONCETE,Nivea,27.35


In [31]:
lpz['total'] = np.where(
    np.abs(lpz['total'] - lpz['total_calculado']) < 0.01,
    lpz['total'],
    lpz['total_calculado']
)

In [32]:
lpz.drop(columns=['total_calculado'], inplace=True)

In [33]:
lpz.head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,Café,10.16,1,20.16,Entregue com Sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações


In [34]:
print(lpz.isnull().sum())

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total              0
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor        3680
marca              0
dtype: int64


In [35]:
lpz.to_csv('vendas_corrigido.csv', index=False)

##### <p align="center"><strong>TRATAMENTO DO VENDEDOR</strong></p>

In [36]:
lpz[lpz.vendedor.isnull()].head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
112,52388,2018-07-30,07:10:51,MIGUEL ANTÔNIO DE DECCO OLIVEIRA,Manteiga,8.77,5,56.85,Pagamento Confirmado,Juiz de Fora,MG,Brasil,36000-000,13.0,Cartão de Crédito,NaN,Aviação
425,46034,2019-12-28,14:14:40,RAFAEL HENRIQUE COSTA BARNABE MARAZO,Suco de Laranja,825.00,5,4125.00,Aguardando Pagamento,Leopoldina,MG,Brasil,367XXXXX,0.0,Boleto,NaN,Prats
525,30444,2022-09-11,21:39:42,LUCAS GONÇALVES DONADONI,Café,10.33,2,20.66,Em Separação,Niterói,RJ,Brasil,24000-000,-0.0,Transferência Bancária,NaN,3 Corações
543,41142,2020-10-16,12:47:08,GABRIEL CARLOS DE SOUZA,Vinho,28.89,1,28.89,Pagamento Confirmado,Coronel Xavier Chaves,MG,Brasil,36330-000,0.0,Cartão de Crédito,NaN,Miolo
544,194,2020-06-28,12:28:55,MATEUS Araujo Kuhn,Suco de Laranja,6.86,6,51.16,Em Separação,Lima Duarte,MG,Brasil,36140-000,10.0,Pix,NaN,Prats
623,55288,2020-05-08,14:08:29,SAMUEL HENRIQUE CAÇADOR,Queijo Mussarela,17.69,1,17.69,Entregue,Niterói,RJ,Brasil,24000-000,0.0,Transferência Bancária,NaN,Quatá


In [37]:
lpz['vendedor'] = lpz['vendedor'].fillna('DESCONHECIDO')

In [38]:
lpz.head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,Café,10.16,1,20.16,Entregue com Sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações


In [39]:
lpz.to_csv('vendas_corrigido.csv', index=False)

##### <p align="center"><strong>TRATAMENTO DE DUPLICATAS</strong></p>

In [40]:
duplicatas = lpz[lpz.duplicated()]
print(duplicatas)

        id_da_compra        data      hora  \
9451           37102  2020-08-02  10:32:27   
11787          43901  2019-07-20  13:10:10   
13994           5337  2020-05-24  12:10:42   
14387           2852  2018-08-26  17:38:01   
14990          40818  2019-03-31  20:04:56   
...              ...         ...       ...   
368476         47230  2019-06-22  16:18:20   
368503         50402  2022-06-25  08:13:33   
368603         31673  2020-12-31  19:24:49   
368628         14688  2019-01-06  18:01:24   
368715         44677  2019-04-22  06:58:43   

                                   cliente         produto  valor  quantidade  \
9451                 JOÃO SILVINO DA SILVA         Shampoo   7.65          12   
11787               KALLEBE RIBEIRO AMORIM    Desinfetante   5.97           6   
13994              GABRIEL VIEIRA DEMARQUE    Desinfetante   6.34           2   
14387           MICAEL SEGURASSE GONÇALVES           Vinho  32.69          11   
14990      TIAGO ANTÔNIO DE DECCO OLIVEIRA

In [41]:
# Remover duplicatas com base em todas as colunas
lpz = lpz.drop_duplicates()

In [42]:
duplicatas = lpz[lpz.duplicated()]
print(duplicatas)

Empty DataFrame
Columns: [id_da_compra, data, hora, cliente, produto, valor, quantidade, total, status, cidade, estado, pais, cep, frete, pagamento, vendedor, marca]
Index: []


In [43]:
lpz.to_csv('vendas_corrigido.csv', index=False)


## <p align="center"><strong><span style="color:red;">Relatório sobre a limpeza!!</span></strong></p>

🔧 Etapas da Limpeza Realizadas
1. Padronização de Tipos e Formatos

    Datas:

        Todas as datas foram convertidas para o formato YYYY-MM-DD.

    Horas:

        Horários convertidos para o padrão HH:MM:SS.

    Valores Numéricos:

        As colunas valor, quantidade, frete e total foram convertidas para tipo numérico (float ou int).

        Valores com símbolos (ex: "R$", ",") foram limpos para manter apenas os números com . como separador decimal.

        Importação da biblioteca (re) para facilitar na limpeza de caracteres nas colunas.

2. Tratamento de Inconsistências

    Valores Ausentes:

        Registros com campos essenciais ausentes (ex: valor, quantidade) foram removidos.

        Dados faltantes não essenciais foram preenchidos com estratégias específicas (ex: frete com média, datas com modo).

    Registros Duplicados:

        Foram identificados e removidos X registros duplicados, mantendo apenas a versão mais recente ou completa.

    Verificação de Cálculo do Total:

        Aplicada a fórmula:
        total = valor * quantidade + frete

        Registros com cálculos incorretos foram corrigidos

    Tratamento do nome do Vendedor:

        Conforme vimos a uma inconsistencia em relaçao ao nome dos vendedores nesse caso colocamos eles como DESCONHECIDO ao realizar a limpeza.
